In [8]:
import numpy as np

class CART:
    def __init__(self, max_depth=7, min_samples_split=2, random_state=42): #initialisation de l'arbre
        self.max_depth = max_depth #attribut de la profondeur max de l'abre (nb max de niveaux de divisions)
        self.min_samples_split = min_samples_split #nombre minimum d’échantillons par nœud pour pouvoir faire une division
        self.random_state = np.random.RandomState(random_state)
        self.feature = None #attribut qui stocke l'indice de la variable choisie pour diviser les données
        self.threshold = None #attribut qui stocke la valeur seuil de la carcatéristique choisie pour diviser les données à ce niveau de l'abre
        self.left = None #sous-arbre gauche
        self.right = None #sous-arbre droit
        self.label = None #stocke la valeur prédite à ce noeud de l'abre
 
    def fit(self, X, y): #Entraîne l'arbre avec les données d'entraînement
        self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):  #Développe l'arbre récursivement en divisant les données en fonction du meilleur gain d'impureté (MSE)
         # Arrêt si la profondeur maximale est atteinte ou si toutes les cibles sont identiques
        if depth >= self.max_depth or len(np.unique(y)) == 1 or len(y) < self.min_samples_split:
            self.label = np.mean(y)
            return

        best_gain = 0
        n_samples, n_features = X.shape
        current_impurity = self._calc_mse(y)

        # Trouver la meilleure séparation
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                y_left = y[X[:, feature] <= threshold]
                y_right = y[X[:, feature] > threshold]

                left_impurity = self._calc_mse(y_left)
                right_impurity = self._calc_mse(y_right)
                impurity_gain = current_impurity - (len(y_left) / n_samples * left_impurity + len(y_right) / n_samples * right_impurity)

                if impurity_gain > best_gain:
                    best_gain = impurity_gain
                    self.feature = feature
                    self.threshold = threshold
        
        # Arrêt si aucun gain n'est réalisé
        if best_gain == 0:
            self.label = np.mean(y)
            return

        # Diviser l'ensemble de données
        left_indices = X[:, self.feature] <= self.threshold
        right_indices = X[:, self.feature] > self.threshold
        self.left = CART(max_depth=self.max_depth, min_samples_split=self.min_samples_split, random_state=self.random_state)
        self.left._grow_tree(X[left_indices], y[left_indices], depth + 1)
        self.right = CART(max_depth=self.max_depth, min_samples_split=self.min_samples_split, random_state=self.random_state)
        self.right._grow_tree(X[right_indices], y[right_indices], depth + 1)

    def _calc_mse(self, y): #Calcule l'erreur quadratique moyenne (MSE)
        return np.mean((y - np.mean(y)) ** 2)

    def predict(self, X): #Applique _predict  pour obtenir les prédictions
        return np.array([self._predict(x) for x in X])

    def _predict(self, x): #Navigue dans l'arbre pour faire une prédiction pour un exemple donné
        if self.feature is not None:
            if x[self.feature] <= self.threshold:
                return self.left._predict(x)
            else:
                return self.right._predict(x)
        else:
            return self.label

    def print_tree(self, depth=0): #Affiche l'arbre de décision de manière structurée
        if self.feature is not None:
            print(f"{'  ' * depth}if X[{self.feature}] <= {self.threshold}:")
            self.left.print_tree(depth + 1)
            print(f"{'  ' * depth}else:")
            self.right.print_tree(depth + 1)
        else:
            print(f"{'  ' * depth}Predict: {self.label}")

